### Mount google drive to the colab

In [1]:
# comment this line if using local machine
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing necessary libraries

In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt

import pickle

# sklearn libraries
from sklearn.model_selection import train_test_split

# tensorflow/keras libraries
import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

## Data preparation and preprocessing 

In [0]:
X_train =0 
y_train=0

i=0
j=1000
X_train = np.load("drive/My Drive/Project/Train/x_mag_500batch1.npy", mmap_mode='r')[i:i+j]
y_train = np.load("drive/My Drive/Project/Train/y_mag_500batch1.npy", mmap_mode='r')[i:i+j]

X_train = np.vstack((X_train,np.load("drive/My Drive/Project/Train/x_mag_500batch2.npy", mmap_mode='r')[i:i+j]))
y_train = np.vstack((y_train,np.load("drive/My Drive/Project/Train/y_mag_500batch2.npy", mmap_mode='r')[i:i+j]))

X_train = np.vstack((X_train,np.load("drive/My Drive/Project/Train/x_mag_500batch3.npy", mmap_mode='r')[i:i+j]))
y_train = np.vstack((y_train,np.load("drive/My Drive/Project/Train/y_mag_500batch3.npy", mmap_mode='r')[i:i+j]))

X_train = np.vstack((X_train,np.load("drive/My Drive/Project/Train/x_mag_500batch4.npy", mmap_mode='r')[i:i+j]))
y_train = np.vstack((y_train,np.load("drive/My Drive/Project/Train/y_mag_500batch4.npy", mmap_mode='r')[i:i+j]))

X_train = np.vstack((X_train,np.load("drive/My Drive/Project/Train/x_mag_500batch1_drone_full_noise.npy", mmap_mode='r')[i:i+j]))
y_train = np.vstack((y_train,np.load("drive/My Drive/Project/Train/y_mag_500batch1.npy", mmap_mode='r')[i:i+j]))

X_train = np.vstack((X_train,np.load("drive/My Drive/Project/Train/x_mag_500batch2_drone_full_noise.npy", mmap_mode='r')[i:i+j]))
y_train = np.vstack((y_train,np.load("drive/My Drive/Project/Train/y_mag_500batch2.npy", mmap_mode='r')[i:i+j]))

X_train = np.vstack((X_train,np.load("drive/My Drive/Project/Train/x_mag_500batch3_drone_full_noise.npy", mmap_mode='r')[i:i+j]))
y_train = np.vstack((y_train,np.load("drive/My Drive/Project/Train/y_mag_500batch3.npy", mmap_mode='r')[i:i+j]))

X_train = np.vstack((X_train,np.load("drive/My Drive/Project/Train/x_mag_500batch4_drone_full_noise.npy", mmap_mode='r')[i:i+j]))
y_train = np.vstack((y_train,np.load("drive/My Drive/Project/Train/y_mag_500batch4.npy", mmap_mode='r')[i:i+j]))

In [4]:
X_train.shape

(8000, 8, 129, 17)

In [0]:
y = np.zeros((8000,1))
y[4000:8000] = 1

#### Creating train test split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=42, stratify=y)

## Denoising model creation (CED)

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'
                 ,data_format= "channels_first",input_shape=(8,129,17)))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2D(64, (5,5),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2DTranspose(128, (1, 1),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2DTranspose(64, (3,3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2DTranspose(64, (3,3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2DTranspose(32, (5,5),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))
model.add(Conv2DTranspose(1, (5, 5),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
model.add(BatchNormalization())#axis=1))
model.add(Activation('relu'))


model.summary()

W0804 20:51:11.092481 140216419866496 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 125, 13)       6432      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 125, 13)       128       
_________________________________________________________________
activation (Activation)      (None, 32, 125, 13)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 121, 9)        51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 121, 9)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 121, 9)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 119, 7)        3

#### Setup callback method for early stopping

In [0]:
early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                       patience=5,
                                       verbose=2,
                                       restore_best_weights=True)


## Model Compilation and training

In [10]:
model.compile(loss=tensorflow.keras.losses.mean_squared_error,
                  optimizer=tensorflow.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999),
                  metrics=['accuracy'])

history = model.fit(X_train,y_train,
                    batch_size=256,
                    epochs=200,
                    validation_split=0.2,
                    shuffle = True,
                    callbacks=[early_stopping_monitor])

Train on 5120 samples, validate on 1280 samples
Epoch 1/200


InvalidArgumentError: ignored

## Saving model and its training history

In [0]:
model.save("drive/My Drive/Project/Models/Denoise/CED/cnn_model_param_348,085_val_loss_7141.9082_val_acc_0.1661_keras.h5")
pickle.dump(model.history.history, open('drive/My Drive/Project/Models/Denoise/CED/history_cnn_model_param_348,085_val_loss_7141.9082_val_acc_0.1661_keras.pkl', 'wb'))

## Loading trained model for prediction and analysis

In [0]:
model = load_model("drive/My Drive/Project/Models/Denoise/CED/cnn_model_param_348,085_val_loss_7141.9082_val_acc_0.1661_keras.h5")

#### Model architecture plot

In [0]:
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
plot_model(model, to_file="drive/My Drive/Project/Denoise/CED/ced_architecture.png")

## Model evalution and analysis

In [0]:
val_loss, val_acc = model.evaluate(X_test, y_test)

### Accuracy Score

In [0]:
print(f'Validation loss of Model : {val_loss}')
print(f'Validation accuracy of Model : {val_acc}')

### Model training vs validation loss plot

In [0]:
pickle.load(open('drive/My Drive/Project/Models/Denoise/CED/history_cnn_model_param_348,085_val_loss_7141.9082_val_acc_0.1661_keras.pkl', 'rb'))

In [0]:
train_loss = history['loss']
train_acc = history['acc']
val_loss = history['val_loss']
val_acc = history['val_acc']

In [0]:
plt.plot(train_loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss (RESNET)')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.savefig("drive/My Drive/Project/Models/Denoise/RESNET/loss_graph_resnet_model.png")
plt.show()